In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import torch
torch.manual_seed(0)
np.random.seed(0)
import torch.nn as nn
from torch.autograd import Variable
import torchmetrics as metrics
from torch.utils.data import Dataset, DataLoader,  TensorDataset
import torch.optim as optim
from torch.nn import functional as F

import random
from patsy import dmatrices
import statsmodels.api as sm

import matplotlib.gridspec as gridspec
# Set the Seaborn context to 'talk' and style to 'whitegrid'
sns.set_context('talk')
sns.set_style('white')

C:\ProgramData\miniconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Read Excel file
excel_file = './weeklydata.xlsx'
df = pd.read_excel(excel_file, engine='openpyxl')

In [3]:
# View data and column headings
print("Excel Data:")
print(df.head())
print("\nColumn Headings:")
print(df.columns)

Excel Data:
    state  year   datetime  epiweek  cases  tempmaxw  tempminw  tempw  \
0  Bauchi  2018 2018-01-07        1    1.0      29.5      10.8   19.0   
1  Bauchi  2018 2018-01-14        2    0.0      27.6      10.8   19.3   
2  Bauchi  2018 2018-01-21        3    0.0      30.5      11.7   21.2   
3  Bauchi  2018 2018-01-28        4    0.0      28.5      13.1   20.4   
4  Bauchi  2018 2018-02-04        5    0.0      34.6      15.6   24.8   

   humidityw  precipw  precipcovw  
0       36.1      0.0         0.0  
1       28.1      0.0         0.0  
2       24.8      0.0         0.0  
3       30.9      0.0         0.0  
4       23.8      0.0         0.0  

Column Headings:
Index(['state', 'year', 'datetime', 'epiweek', 'cases', 'tempmaxw', 'tempminw',
       'tempw', 'humidityw', 'precipw', 'precipcovw'],
      dtype='object')


In [4]:
assert df.isnull().sum().sum() == 0

In [5]:
feature_columns = ['tempmaxw', 'tempminw', 'tempw', 'humidityw', 'precipw', 'precipcovw', 'cases']
feature_scaler = MinMaxScaler(feature_range=(0, 1))

df[feature_columns] = feature_scaler.fit_transform(df[feature_columns].values)
df.head()

,state,year,datetime,epiweek,cases,tempmaxw,tempminw,tempw,humidityw,precipw,precipcovw
0,Bauchi,2018,2018-01-07,1,0.019608,0.419890,0.089385,0.088435,0.308126,0.0,0.0
1,Bauchi,2018,2018-01-14,2,0.000000,0.314917,0.089385,0.108844,0.217833,0.0,0.0
2,Bauchi,2018,2018-01-21,3,0.000000,0.475138,0.139665,0.238095,0.180587,0.0,0.0
3,Bauchi,2018,2018-01-28,4,0.000000,0.364641,0.217877,0.183673,0.249436,0.0,0.0
4,Bauchi,2018,2018-02-04,5,0.000000,0.701657,0.357542,0.482993,0.169300,0.0,0.0


In [6]:
def create_dataset(dataset, features_column, lookback):
    X, y = [], []
    # print(dataset[features_column].head())
    for i in range(dataset.shape[0]-lookback):
        feature = dataset[features_column].iloc[i:i+lookback].values
        target = dataset[features_column].iloc[i+1:i+lookback+1].values
        X.append(feature)
        y.append(target)
    return torch.tensor(X, dtype=torch.float), torch.tensor(y, dtype=torch.float)    

In [7]:
loopback = 4
X, Y = create_dataset(df, feature_columns, loopback)
print(X[2])
print(Y[1])

tensor([[0.4751, 0.1397, 0.2381, 0.1806, 0.0000, 0.0000, 0.0000],
        [0.3646, 0.2179, 0.1837, 0.2494, 0.0000, 0.0000, 0.0000],
        [0.7017, 0.3575, 0.4830, 0.1693, 0.0000, 0.0000, 0.0000],
        [0.7569, 0.4246, 0.6259, 0.1467, 0.0000, 0.0000, 0.0392]])
tensor([[0.4751, 0.1397, 0.2381, 0.1806, 0.0000, 0.0000, 0.0000],
        [0.3646, 0.2179, 0.1837, 0.2494, 0.0000, 0.0000, 0.0000],
        [0.7017, 0.3575, 0.4830, 0.1693, 0.0000, 0.0000, 0.0000],
        [0.7569, 0.4246, 0.6259, 0.1467, 0.0000, 0.0000, 0.0392]])


C:\Users\MSI\AppData\Local\Temp\ipykernel_17024\898535213.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  return torch.tensor(X, dtype=torch.float), torch.tensor(y, dtype=torch.float)


In [8]:
class EpidemiologyModel(nn.Module):
    def __init__(self, feature_dim=8, hidden_size=128, num_lstm_layers = 2, bidirection=False, device='cpu'):
        super().__init__()
        self.device = device
        self.bidirection = bidirection
        self.hidden_size = hidden_size
        self.num_lstm_layers = num_lstm_layers
        self.lstm = nn.LSTM(input_size=feature_dim, hidden_size=self.hidden_size, dropout=0.2, bidirectional=self.bidirection, num_layers=self.num_lstm_layers, batch_first=True, device=self.device)
        if bidirection:
            self.regressor = nn.Linear(2*self.hidden_size, feature_dim, device= self.device)
        else:
            self.regressor = nn.Linear(self.hidden_size, feature_dim, device= self.device)
        self.relu = m = F.relu
    def forward(self, x):
        self.lstm.flatten_parameters()
        if self.bidirection:
            h_0 = Variable(torch.zeros(2*self.num_lstm_layers, x.size(0), self.hidden_size, device= self.device)) #hidden state
            c_0 = Variable(torch.zeros(2*self.num_lstm_layers, x.size(0), self.hidden_size, device = self.device)) #internal state
        else:
            h_0 = Variable(torch.zeros(self.num_lstm_layers, x.size(0), self.hidden_size, device = self.device)) #hidden state
            c_0 = Variable(torch.zeros(self.num_lstm_layers, x.size(0), self.hidden_size, device= self.device)) #internal state

        out, (h_n, c_o) = self.lstm(x, (h_0, c_0))  # output, (hidden, cell_state)
        out = self.relu(out)
        x = self.regressor(out)
        
        return x
        
class EarlyStopping:
    def __init__(self, tolerance=5):
        self.tolerance = tolerance
        self.min_delta = -torch.inf
        self.counter = 0
        self.stop = False

    def __call__(self, train_loss, validation_loss):        
        delta = abs(validation_loss - train_loss)
        # print("The delta", delta, "min delta ", self.min_delta)
        if delta > self.min_delta:            
            self.min_delta = delta            
            self.counter +=1
            # print("counter", self.counter)
            if self.counter >= self.tolerance:
                self.stop = True
        else:
            self.counter = 0


In [9]:
def train(df, train_val_years, feature_columns, lstm_hidden_size=100, n_lstm_layers=2,
          lookback = 6, n_epochs = 2000, bidirection=False,
          enable_early_Stop =True, early_stop_tolerance = 10, state_filter='Bauchi'):

    

    df = df[df['year'].isin(train_val_years)]    
    
    data = df[df['state']==state_filter].copy()
    data.sort_values(['year', 'epiweek'], inplace=True)
    data.sort_values(['year', 'epiweek'], inplace=True)


    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = EpidemiologyModel(feature_dim=len(feature_columns), hidden_size=lstm_hidden_size, num_lstm_layers=n_lstm_layers, device=device)
    model = model.to(device)
    
    optimizer = optim.Adam(model.parameters())
    loss_fn = nn.MSELoss()

    data.index = data.datetime
    data.drop(columns=['state', 'year' ,'datetime', 'epiweek'], inplace=True)
       

    X, Y = create_dataset(data, feature_columns, lookback)

    X, Y = X.to(device), Y.to(device)


    train_len = int(X.shape[0]*0.80)
    X_train, Y_train, X_val, Y_val = X[:train_len], Y[:train_len], X[train_len:], Y[train_len:]    
    
    
    loader = DataLoader(TensorDataset(X_train, Y_train), shuffle=False, batch_size=32)

    early_stopping = EarlyStopping(tolerance=early_stop_tolerance)
    train_log = []
    val_log = []
    print("Training Model for {} ....".format(state_filter))
    for epoch in range(n_epochs):
        model.train()
        for X_batch, Y_batch in loader:
            Y_pred = model(X_batch)
            loss = loss_fn(Y_pred, Y_batch) + 0.4*F.relu(-Y_pred).mean()
            
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            model.eval()
            train_rmse = np.sqrt(loss_fn(Y_pred, Y_batch).cpu().item())
            train_log.append(train_rmse)

            Y_pred_val = model(X_val)
            val_rmse = np.sqrt(loss_fn(Y_pred_val, Y_val).cpu().item())
            val_log.append(val_rmse)
            # early_stopping(val_rmse)
            early_stopping(train_rmse, val_rmse)
            
            if enable_early_Stop and early_stopping.stop:
                print("We stopped at epoch:", epoch)
                break
            if (epoch+1) % 10 == 0:
                print("Epoch %d: train MSE %.4f, val MSE %.4f" % (epoch, train_rmse, val_rmse))
    print("Training Model for {a} completed with Test RMSE {b:.2f}".format(a=state_filter, b=val_rmse))
    fig= plt.figure(figsize=(20,10))
    plt.plot(np.array(train_log), c ='red', label="Train")
    plt.plot(np.array(val_log), c ='green', label="Val")
    plt.xlabel("Epochs")
    plt.ylabel("MSE")
    plt.legend()

    plt.title('Training and Test Performance for {a} state.\n Test RMSE {b:.2f}'.format(a=state_filter, b=val_rmse), fontsize=20)
    plt.tight_layout()
    plt.savefig('{}_train_viz.svg'.format(state_filter), bbox_inches='tight')
    return model

In [10]:
# Define the years for training and testing data
train_val_years = [2018, 2019, 2020, 2021, 2022]  # Years for training data
test_years = [2018, 2019, 2020, 2021, 2022, 2023]  # test behavior of all years including 2023

lookback = 4

In [11]:
from ipywidgets import interact_manual
@interact_manual(retrain_model = [False, True], state=["Bauchi", "Edo", "Ondo"], lookback = [4, 8, 1], lstm_hidden_size=[30, 40, 60]
                 , n_lstm_layers=[3, 2, 4, 1], bidirection_lstm=[False, True]
                 , n_epochs = [1000, 1500, 2000], enable_early_Stop = {False,True}, early_stop_tolerance = [5, 10, 15, 20])
def train_visualization(retrain_model,n_epochs, state, lookback, lstm_hidden_size, n_lstm_layers
                  ,bidirection_lstm, enable_early_Stop, early_stop_tolerance):
    save_path = "./{}_model.pt".format(state)
    
    state_filter = state
    data=df
    if retrain_model:
        model = train(df, train_val_years, feature_columns, lstm_hidden_size=lstm_hidden_size, n_lstm_layers=n_lstm_layers, lookback = lookback,
                           n_epochs = n_epochs, enable_early_Stop = True, early_stop_tolerance = early_stop_tolerance, bidirection=False, state_filter=state_filter)
        torch.save(model, save_path)
    else:
        model = torch.load(save_path, weights_only=False)

    mae = metrics.MeanAbsoluteError()
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    data = df[df['state']==state_filter]
    data = data[data['year'].isin(test_years)].copy()
    data.sort_values(['year', 'epiweek'], inplace=True)
    data.sort_values(['year', 'epiweek'], inplace=True)
    

    
    test_gp = data.groupby(['state'])
    s = state_filter
    state_data = test_gp.get_group(s)
    fig, axs = plt.subplots(2, 3, figsize=(20,10))
    fig_future, axs_future = plt.subplots(1, 2, figsize=(20,10))
    for j, ys in enumerate(test_years):
        state_data_year = state_data[state_data['year']== ys]
        state_data_year.drop(columns=['state', 'year', 'datetime'], inplace=True)
        X_test, y_test = create_dataset(state_data_year, feature_columns, lookback)

        X_test, y_test = X_test.to(device), y_test.to(device)

        with torch.no_grad():
          model.eval()
          y_pred = model(X_test)
          err= mae(y_pred, y_test).item()
          y_pred_transformed = feature_scaler.inverse_transform(y_pred.cpu().numpy()[:, -1, :])
          y_test_transformed = feature_scaler.inverse_transform(y_test.cpu().numpy()[:, -1, :])

        #
       # plot
        axs[j//3, j%3].title.set_text(str(ys)+", Mean Absolute error (MAE): %.2f"%(err))
        axs[j//3, j%3].plot(y_pred_transformed[:,-1], label='pred_y')
        axs[j//3, j%3].plot(y_test_transformed[:,-1], label='true_y')
        axs[j//3, j%3].legend(loc="upper right")
        axs[j//3, j%3].set_xlabel('Week Index')
        axs[j//3, j%3].set_ylabel('Number of cases')

    ## predicting future years
    for f_year in range(2):            
        with torch.no_grad():
            #first plot the prediction for the last test year
            future_transform = feature_scaler.inverse_transform(y_pred.cpu()[:, -1, :].numpy())
            axs_future[f_year].title.set_text("Year "+str(ys+f_year+1) +" Cases")
            axs_future[f_year].plot(future_transform[:,-1], label='Cases')
            axs_future[f_year].legend()
            axs_future[f_year].set_xlabel('Week Index')
            axs_future[f_year].set_ylabel("Cases")
            y_pred = model(y_pred)


    fig.suptitle('Predictions for {} state'.format(s), fontsize=20)
    fig.tight_layout()
    fig_future.suptitle('Future Predictions for {} state'.format(s), fontsize=20)
    fig.savefig('{}_predictions.svg'.format(state), bbox_inches='tight')
    fig_future.tight_layout()
    fig_future.savefig('{}_future_predictions.svg'.format(state), bbox_inches='tight')
    

interactive(children=(Dropdown(description='retrain_model', options=(False, True), value=False), Dropdown(desc…

In [12]:
from ipywidgets import interact
import shap

class ModelWrapper(nn.Module):
    def __init__(self, theModel, feature_columns, feature_transform , lookback):
        super().__init__()
        self.model = theModel
        self.feature_columns = feature_columns
        self.lookback = lookback
        self.feature_transform =feature_transform 
    def forward(self, X):
        X_test, _ = create_dataset(X, self.feature_columns, self.lookback)
        Pred = feature_scaler.inverse_transform(self.model(X_test).detach().numpy()[:, -1, :])

        for i in range(self.lookback):
            next_series = feature_scaler.inverse_transform(self.model(X_test[[-1]]).detach().numpy()[:, -1, :])
            Pred = np.concatenate((Pred, next_series))
        
        return Pred[:,  -1]

@interact_manual(state=["Bauchi", "Edo", "Ondo"], year = [2018, 2019, 2020, 2021, 2022, 2023], lookback = [4, 8, 1], Num_Instances = [20, 30, 40, 50])
def explain_by_permutation(state, year, Num_Instances):
    data = df[df['state']==state].copy()
    test_data = data[data['year'].isin([year])].copy()
    test_data.sort_values(['year', 'epiweek'], inplace=True)
    test_data.sort_values(['year', 'epiweek'], inplace=True)
    test_data = test_data.reset_index()
    test_data.drop(columns=['state', 'year', 'datetime', 'index', 'epiweek'], inplace=True)    

    save_path = "./{}_model.pt".format(state)
    model = torch.load(save_path, weights_only=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = torch.load(save_path, weights_only=False)
    model = model.to(device)
    model = ModelWrapper(model,feature_columns, feature_scaler, lookback)

    explainer = shap.PermutationExplainer(model, test_data[:Num_Instances])
    shap_values = explainer(test_data[:Num_Instances])
    shap.plots.heatmap(shap_values, feature_values=shap_values.abs.max(0), show=False)    
    plt.title('{} state SHAP plot for the year {}'.format(state, year))
    plt.savefig('{}_shap_plot.svg'.format(state), bbox_inches='tight')



interactive(children=(Dropdown(description='state', options=('Bauchi', 'Edo', 'Ondo'), value='Bauchi'), Dropdo…

In [13]:
from ipywidgets import interact_manual
@interact_manual(state=["Bauchi", "Edo", "Ondo"], year = [2018, 2019, 2020, 2021, 2022, 2023]
          , addition_viz = {'Max Weekly Temp':('Max Weekly Temp', -7)
                            , 'Min Weekly Temp':('Min Weekly Temp',-6), 'Weekly of Temp':('Weekly Temp', -5)
                            , 'Weekly Humidity': ('Weekly Humidity', -4), 'Weekly Precipitation':('precipw', -3)
                            , 'Weekly Precipcovw': ('Weekly Precipcovw', -2)
                            }, lookback = [4, 8, 1])
def visualization_compare(state, addition_viz, year, lookback):
    save_path = "./{}_model.pt".format(state)
    
    state_filter = state
    data=df
    model = torch.load(save_path, weights_only=False)
    
    mae = metrics.MeanAbsoluteError()
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    data = df[df['state']==state_filter]
    data = data[data['year'].isin(test_years)].copy()
    data.sort_values(['year', 'epiweek'], inplace=True)
    data.sort_values(['year', 'epiweek'], inplace=True)
    

    
    test_gp = data.groupby(['state'])
    s = state_filter
    state_data = test_gp.get_group(s)
    fig_cases_addviz, axs_cases_addviz = plt.subplots(1, 2, figsize=(15,5))
  
    
    ys = year
    state_data_year = state_data[state_data['year']== ys]
    state_data_year.drop(columns=['state', 'year', 'datetime'], inplace=True)
    X_test, y_test = create_dataset(state_data_year, feature_columns, loopback)

    X_test, y_test = X_test.to(device), y_test.to(device)

    with torch.no_grad():
        model.eval()
        y_pred = model(X_test)   
        err= mae(y_pred, y_test).item()
        
        y_pred_transform = feature_scaler.inverse_transform(y_pred.cpu()[:, -1, :].numpy())
        y_test_transform = feature_scaler.inverse_transform(y_test.cpu()[:, -1, :].numpy())

    axs_cases_addviz[0].title.set_text(str(ys)+" Cases. MAE = %.2f"%(err))
    axs_cases_addviz[0].plot(y_pred_transform[:,-1], label='pred_y')
    axs_cases_addviz[0].plot(y_test_transform[:,-1], label='true_y')
    axs_cases_addviz[0].set_xlabel('Week Index')
    axs_cases_addviz[0].set_ylabel('Number of cases')
    axs_cases_addviz[0].legend()
    
    axs_cases_addviz[1].title.set_text(str(ys)+" {1}. MAE = {0:.2f}".format(err, addition_viz[0]))
    axs_cases_addviz[1].plot(y_pred_transform[:,addition_viz[1]], label='pred_y')
    axs_cases_addviz[1].plot(y_test_transform[:,addition_viz[1]], label='true_y')            
    axs_cases_addviz[1].set_xlabel('Week Index')
    axs_cases_addviz[1].set_ylabel(addition_viz[0])
    axs_cases_addviz[1].legend()          

    
    fig_cases_addviz.suptitle('{a} Predictions for {b} state'.format(b=s, a=year), fontsize=20)    
    fig_cases_addviz.tight_layout()
    
    plt.tight_layout()
    plt.savefig('{}_{}_{}_predict_compare.svg'.format(state, year, addition_viz[0]), bbox_inches='tight')
    


interactive(children=(Dropdown(description='state', options=('Bauchi', 'Edo', 'Ondo'), value='Bauchi'), Dropdo…

In [14]:
from ipywidgets import interact_manual
@interact_manual(state=["Bauchi", "Edo", "Ondo"]
          , addition_viz = {'Max Weekly Temp':('Max Weekly Temp', -7)
                            , 'Min Weekly Temp':('Min Weekly Temp',-6), 'Weekly of Temp':('Weekly Temp', -5)
                            , 'Weekly Humidity': ('Weekly Humidity', -4), 'Weekly Precipitation':('precipw', -3)
                            , 'Weekly Precipcovw': ('Weekly Precipcovw', -2)
                            }, lookback = [4, 8, 1])
def visualization_compare_all(state, addition_viz, lookback):
    save_path = "./{}_model.pt".format(state)
    
    state_filter = state
    data=df
    model = torch.load(save_path, weights_only=False)
    
    mae = metrics.MeanAbsoluteError()
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    data = df[df['state']==state_filter]
    data = data[data['year'].isin(test_years)].copy()
    data.sort_values(['year', 'epiweek'], inplace=True)
    data.sort_values(['year', 'epiweek'], inplace=True)
    

    
    test_gp = data.groupby(['state'])
    s = state_filter
    state_data = test_gp.get_group(s)
    fig_cases_addviz, axs_cases_addviz = plt.subplots(6, 2, figsize=(15,25))
  
    fig_future, axs_future = plt.subplots(2,1, figsize=(15,20))
    
    for j, ys in enumerate(test_years):
        state_data_year = state_data[state_data['year']== ys]
        state_data_year.drop(columns=['state', 'year', 'datetime'], inplace=True)
        X_test, y_test = create_dataset(state_data_year, feature_columns, loopback)

        X_test, y_test = X_test.to(device), y_test.to(device)

        with torch.no_grad():
            model.eval()
            y_pred = model(X_test)   
            err= mae(y_pred, y_test).item()
            
            y_pred_transform = feature_scaler.inverse_transform(y_pred.cpu()[:, -1, :].numpy())
            y_test_transform = feature_scaler.inverse_transform(y_test.cpu()[:, -1, :].numpy())

        axs_cases_addviz[j, 0].title.set_text(str(ys)+" Cases. MAE = %.2f "%(err))
        axs_cases_addviz[j, 0].plot(y_pred_transform[:,-1], label='pred_y')
        axs_cases_addviz[j, 0].plot(y_test_transform[:,-1], label='true_y')
        axs_cases_addviz[j, 0].set_xlabel('Week Index')
        axs_cases_addviz[j, 0].set_ylabel('Number of cases')
        axs_cases_addviz[j, 0].legend()
        
        axs_cases_addviz[j, 1].title.set_text(str(ys)+" {1}. MAE = {0:.2f}".format(err, addition_viz[0]))
        axs_cases_addviz[j, 1].plot(y_pred_transform[:,addition_viz[1]], label='pred_y')
        axs_cases_addviz[j, 1].plot(y_test_transform[:,addition_viz[1]], label='true_y')            
        axs_cases_addviz[j, 1].set_xlabel('Week Index')
        axs_cases_addviz[j, 1].set_ylabel(addition_viz[0])
        axs_cases_addviz[j, 1].legend()          

        ## predicting future years
    for f_year in range(2):            
        with torch.no_grad():
            #first plot the prediction for the last test year
            future_transform = feature_scaler.inverse_transform(y_pred.cpu()[:, -1, :].numpy())
            # axs_future[f_year].title.set_text("Year "+str(ys+f_year+1) +" Cases Vs {}".format(addition_viz[0]))
            axs_future[f_year].title.set_text("Year "+str(ys+f_year+1) +" Cases")
            axs_future[f_year].plot(future_transform[:,-1], label='Cases')
            # axs_future[f_year].plot(future_transform[:,addition_viz[1]], label= addition_viz[0])
            axs_future[f_year].legend()
            axs_future[f_year].set_xlabel('Week Index')
            # axs_future[f_year].set_ylabel("Cases/"+addition_viz[0])
            axs_future[f_year].set_ylabel("Cases")
            y_pred = model(y_pred)
            

    
    fig_cases_addviz.suptitle('Predictions for {} state'.format(s), fontsize=20)
    fig_future.suptitle('Future Predictions for {} state'.format(s), fontsize=20)
    
    fig_cases_addviz.tight_layout()
    fig_future.tight_layout()        
    plt.tight_layout()
    
    fig_cases_addviz.savefig('{}_{}_predict_compare_all_years.svg'.format(state, addition_viz[0]), bbox_inches='tight')
    fig_future.savefig('{}_future_prediction.svg'.format(state), bbox_inches='tight')

interactive(children=(Dropdown(description='state', options=('Bauchi', 'Edo', 'Ondo'), value='Bauchi'), Dropdo…